## 1. write a function which will try to find out len of a string without using an inbuilt len function 

In [1]:
def strLen(str):
    ctr = 0
    
    for i in str:
        ctr += 1
    
    return ctr


str = "Hello World"
print(strLen(str))

11


## 2. write a function which  will be able to print an index of list element without using an index function

In [21]:
def getIndex(lst, elem):
    

    for i in range(len(lst)):
        val = lst[i]

        if val == elem:
            return i

    return None

list1 = [10,20,30,40]
print(f"Index: {getIndex(list1, 50)}")

Index: None


## 3. write a function which will be able to print an ip address of your system

In [24]:
def getIP():

    import socket
    
    ip = socket.gethostbyname(socket.gethostname())
    
    return ip

getIP()

'127.0.0.1'

## 4. write a function which will shutdown your system

In [1]:
def sys_shutdown():
    resp = input("Shutdown system? Y/N")
    
    if resp == "Y":
        os.system("shutdown /s /t 1")
    else:
        return

sys_shutdown()

Shutdown system? Y/N N


## 5. write a function which will take input as a list with any kind of numeric value and give an out as a multiplication of all the numeric data l = [3.5, 6.56, 4,5,"sudh" , "ineuron" , 'fsda bootcamp 2.0']

In [8]:
def multi(lst):
    ctr = 1
    
    for i in lst:
        if type(i) == int or type(i) == float:
            ctr = i*ctr
    
    return ctr

lst =  [3.5, 6.56, 4,5,"sudh" , "ineuron" , 'fsda bootcamp 2.0']
print(multi(lst))


459.19999999999993


## 6. write a function which will be able to read all the mails 

Here I am attempting to use Gmail API using OAuth 2.0 which is the recommended method by Google. Code is used from their Quickstart tutorial but modified slightly so I can only read email headers of latest 2 messages and not the entire message.

This method makes it more secure as unlike App Password method mentioned in the class, this one requires a token file "credentials.json" which if not available, the code will not work.

In [11]:
# Install Gmail Client API libraries

!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.9 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 9.1 MB/s eta 0:00:00
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 15.3 MB/s eta 0:00:00
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)


In [18]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def read_mail():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me').execute()
#         labels = results.get('labels', [])

#         if not labels:
#             print('No labels found.')
#             return
#         print('Labels:')
#         for label in labels:
#             print(label['name'])
        msgs = results.get("messages")
        if not msgs:
            print("No messages found.")
            return
        
        # Read the latest 2 email messages
        
        for msg in msgs[-2:]:
            try:
                txt = service.users().messages().get(userId='me', id=msg['id']).execute()

                payload = txt['payload']
                headers = payload['headers']

                for d in headers:
                    # print(d)
                    if d['name'] == 'Subject':
                        subject = d['value']
                    if d['name'] == 'From':
                        sender = d['value']
                    if d["name"] == "Received":
                        recd_on = d["value"][-37:]   # read last 37 bytes from the value to get the received date
                
                
                print("Subject: ", subject)
                print("From: ", sender)
                print("Received On:", recd_on)

                # parts = payload.get('parts')[0]
                # data = parts['body']['data']
                # data = data.replace("-","+").replace("_","/")
                # decoded_data = base64.b64decode(data)

                # print("Message: ", decoded_data)
                print('\n')
            except:
                continue

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')


read_mail()

Subject:  Artificial Intelligence #156
From:  Andriy Burkov via LinkedIn <newsletters-noreply@linkedin.com>
Received On: Sat, 28 Jan 2023 13:15:35 -0800 (PST)


Subject:  Esther Crawford ✨ Tweeted: I was sent the link to a Turkish news sto...
From:  Twitter <info@twitter.com>
Received On: Sat, 28 Jan 2023 13:15:22 -0800 (PST)




## 7 . write a function which will be able to send a mail to anyone 

Send mail using Gmail OAuth 2.0 method using Gmail Python quickstarts.



In [11]:
import base64
from email.mime.text import MIMEText
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from requests import HTTPError
SCOPES = [
        "https://www.googleapis.com/auth/gmail.send"
    ]
flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
creds = flow.run_local_server(port=0)

service = build('gmail', 'v1', credentials=creds)
message = MIMEText('Sample body of email. Pls. ignore.')
message['to'] = 'mustansir.husain@gmail.com'
message['subject'] = 'iNeuron FSDS 2.0 email sending assignment - Test email'
create_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}
try:
    message = (service.users().messages().send(userId="me", body=create_message).execute())
    print(F'sent message to {message} Message Id: {message["id"]}')
except HTTPError as error:
    print(F'An error occurred: {error}')
    message = None

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=606282883988-qpbf9pi2capkdf31m52lkkf4dsj3t5h0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55845%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=FzsGFKS7xILSHEQoVWVypsnMNTbpDJ&access_type=offline
sent message to {'id': '18601d03ce6cc5bb', 'threadId': '18601d03ce6cc5bb', 'labelIds': ['UNREAD', 'SENT', 'INBOX']} Message Id: 18601d03ce6cc5bb


Mail received is quoted below:

```
Message ID	<CAGx9vjy-jh8Apsp6k-1NvnUTtEy6SVOrrG1xtWU=Rcv5eA4iUQ@mail.gmail.com>

Created at:	Mon, Jan 30, 2023 at 12:34 PM (Delivered after 0 seconds)

From:	mustansir.husain@gmail.com

To:	mustansir.husain@gmail.com

Subject:	iNeuron FSDS 2.0 email sending assignment - Test email
```


## 8. write a function which will be able to read a doc/word file from your system 

In [21]:
# Install package Docx2Txt for reading Word documents

!pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=4e10c24e62e26b4a5f600d3fcf09b16b9aff877d7a38f913204c754189d333fc
  Stored in directory: /Users/mhusain/Library/Caches/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt


In [25]:
import docx2txt

# read document
result = docx2txt.process("document.docx")

print(result)

Sample document.

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

This text is inside the MS Word document.
